In [2]:
%run classesDefinations.ipynb

In [3]:
import functools

import gymnasium
import numpy as np
from gymnasium.spaces import Discrete, Dict, Box
from gymnasium.wrappers import FlattenObservation #IMPORTANT USE IT TO FLATTEN OBS SPACE

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers

import json
import random
import requests

In [4]:
NUMBEROFBUSES = 10
CAPACITY = 5
VEHILECORDSLAT = 35.8942679
VEHILECORDSLOG = 14.5086503 #CORDS OF THE 15 VALLETTA BUS BAY

STARTINGCORD = 1
DESTINATIONCORD = 0

REQUEST_ACCEPTED = 0
REQUEST_REJECTED = 1
REQUEST_DROPPED = 2

LATITUDE_OBSERVATION_INDEX = 0
LONGITUDE_OBSERVATION_INDEX = 1
STARTING_OBSERVATION_INDEX = 2
REQUEST_INDEX = 3

PADDINGVAL = -1

dataFilePath = "./Data/BusStopsMalta/export.json"
OSRM_HOST = "localhost"  # ROSRM server IP or hostname
OSRM_PORT = "5000"       # Port of OSRM server is listening on

In [20]:
from datetime import datetime, timedelta

def divide_time_interval(n):
    """
    Divides the time interval between 8:00 AM and Midnight into 'n' equal intervals.

    Parameters:
    n (int): Number of intervals to divide the time into.

    Returns:
    list: A list of time intervals.
    """
    
    # Define start and end times
    start_time = datetime.strptime("08:00 AM", "%I:%M %p")
    end_time = datetime.strptime("12:00 AM", "%I:%M %p")

    # Calculate total duration in seconds
    total_duration = (end_time - start_time).seconds

    # Calculate duration of each interval in seconds
    interval_duration = total_duration / n

    # Generate time intervals
    intervals = []
    for i in range(n):
        interval_start = start_time + timedelta(seconds=i * interval_duration)
        intervals.append(interval_start.time())

    return intervals

def getTimeIntervalSeconds(interval1, interval2):
    today = datetime.now().date()
    datetime1 = datetime.combine(today, interval1)
    datetime2 = datetime.combine(today, interval2)

    difference = datetime1 - datetime2
    return difference.seconds

def getDistanceBetweenTwoPoints(cord1,cord2):
    start_coords = str(cord1.longitude) + "," + str(cord1.latidude)
    end_coords = str(cord2.longitude) + "," + str(cord2.latidude)
    url = f"http://{OSRM_HOST}:{OSRM_PORT}/route/v1/driving/{start_coords};{end_coords}?overview=false"
    response = requests.get(url)
    # Parsing the response
    if response.status_code == 200:
        data = response.json()
        distance = data["routes"][0]["distance"]
        return distance
    else:
        Exception("Failed to get a response from the OSRM server")
        return None

In [9]:
class BusRoutingHandler():
    def __init__(self, vehileAmount = 10, numberOfRequest = 50):
        self.vehiles = self.initVehiles(vehileAmount)
        self.requests = self.initRequests(numberOfRequest)
        self.currentRequestIndex = 0
        self.currentRequest = self.requests[self.currentRequestIndex]
        self.dropRequests = []
        self.rejectedRequests = []
        self.dropRejectedRequests = []
        self.firstRequest = True

    def __str__ (self):
        stringOutput = "==========================================\n Vehiles: \n"
        for vehile in self.vehiles:
            stringOutput += str(vehile) + "\n"
        stringOutput += "==========================================\n Requests: \n"
        for request in self.requests:
            stringOutput += str(request) + "\n"
        stringOutput += "==========================================\n"
        return stringOutput
    
    def __repr__ (self):
        return self.__str__()
    
    def initVehiles(self, vehileAmount):
        vehileList = []
        for i in range(vehileAmount):
            vechCords = Cords(VEHILECORDSLAT, VEHILECORDSLOG)
            vehileList.append(Vehicle(CAPACITY, vechCords))
        return vehileList

    def initRequests(self, numberOfRequest):
        timeIntervals = divide_time_interval(numberOfRequest)
        f = open(dataFilePath, "r")
        data = json.load(f)
        elements = data["elements"]
        elementsLen = len(elements)
        request = []
        time = 0
        for i in range(numberOfRequest):
            #Get two random indexes from all the bus stops 
            r1 = random.randint(0, elementsLen-1)
            r2 = random.randint(0, elementsLen-1)
            while r1 == r2:
                r2 = random.randint(0, elementsLen-1)
            
            #Get the coordinates of the two bus stops
            lat1 = elements[r1]["lat"]
            lon1 = elements[r1]["lon"]
            lat2 = elements[r2]["lat"]
            lon2 = elements[r2]["lon"]

            #Create the request Cords
            reqPickup = RequestCords(lat1, lon1, True)
            reqDropoff = RequestCords(lat2, lon2, False)

            #Create the request
            time += 20
            request.append(Request(random.randint(1,3), reqPickup, reqDropoff, timeIntervals[i]))
        f.close()
        return request
    
    def updateRequest(self): #Updates the current request to the next one and handles all the logic behind it
        
        #Get the time difference between the current request and the next one
        nextRequest = self.requests[self.currentRequestIndex + 1]
        timeDifference = getTimeIntervalSeconds(nextRequest.getTime(),self.currentRequest.getTime())

        #For each bus that update the route
        for bus in self.vehiles:
            bus.updateRoute(timeDifference)

        #Update the current request
        self.currentRequestIndex += 1
        self.currentRequest = self.requests[self.currentRequestIndex]
        return self.currentRequest
    
    def addRequestToRoute(self, index, request):
        self.vehiles[index].addRequestToRoute(request)

    def dropRequest(self, vehileIndex, requestIndex):
        requestDropped = self.vehiles[vehileIndex].dropRequest(requestIndex)
        self.dropRequests.append(requestDropped)

    def getVehiles(self):
        return self.vehiles

    def getRequests(self):
        return self.requests
    
    def getCurrentRequest(self): #Updates the current request to the next one and handles all the logic behind it
        return self.currentRequest
    
    def getBusCords(self, index):
        busPos = self.vehiles[index].getPosition()
        #Transfrom into an NP Array
        cords = np.array([busPos.getLatitude(), busPos.getLongitude()], dtype=np.float32)
        return cords
    
    def getBusPassengerAmount(self, index):
        return self.vehiles[index].getPassengerAmount()
    
    def getCurrentRequestPosition(self):
        pickUpCords = self.currentRequest.getOrigin()
        dropOffCords = self.currentRequest.getDestination()
  
        #Transform the cords in into a 2D NP array
        cords = np.array([[pickUpCords.getLatitude(), pickUpCords.getLongitude(), STARTINGCORD], [dropOffCords.getLatitude(), dropOffCords.getLongitude(), DESTINATIONCORD]], dtype=np.float32)
        return cords
    
    def getCurrentRequestPassengerAmount(self):
        return self.currentRequest.getPassengerAmount()
    
    def getCurrentRoute(self, index):
        route = self.vehiles[index].getRoute()
        routeSize = route.getSize()

        routeCordsList = route.getListOfCords()

        routeCords = np.zeros((self.vehiles[index].getBusCapacity, 4), dtype=np.float32)

        for i in range(routeSize):
            routeCords[i][LATITUDE_OBSERVATION_INDEX] = routeCordsList[i].getLatitude()
            routeCords[i][LONGITUDE_OBSERVATION_INDEX] = routeCordsList[i].getLongitude()
            routeCords[i][STARTING_OBSERVATION_INDEX] = routeCordsList[i].getStart()
            routeCords[i][REQUEST_INDEX] = routeCordsList[i].getRequestId()
            
        if routeSize < CAPACITY:
            for i in range(routeSize, CAPACITY):
                routeCords[i][LATITUDE_OBSERVATION_INDEX] = PADDINGVAL
                routeCords[i][LONGITUDE_OBSERVATION_INDEX] = PADDINGVAL
                routeCords[i][STARTING_OBSERVATION_INDEX] = PADDINGVAL
                routeCords[i][REQUEST_INDEX] = PADDINGVAL
        return routeCords
    
    def getRouteDistance(self, index):
        return self.vehiles[index].routeDistance()
    
    def getAcceptRequestMask(self, index, requestAcceptedFlag, requestPassengers, routeNPArray):
        #Create an np array of zeros 
        mask = np.zeros((3,), dtype=np.int8)

        mask[REQUEST_REJECTED] = 1 #Request can be rejected by default

        if (self.vehicles[index].getPassengerAmount() + requestPassengers <= CAPACITY) and requestAcceptedFlag == False:
            mask[REQUEST_ACCEPTED] = 1 #Request can be accepted if there is enough space and the request is not already accepted
        
        #Check if there is a request that can be dropped
        for i in range(routeNPArray.shape[0]):
            if routeNPArray[i][0] != PADDINGVAL and routeNPArray[i][1] != PADDINGVAL:
                if routeNPArray[i][STARTING_OBSERVATION_INDEX] == STARTINGCORD:
                    mask[REQUEST_DROPPED] = 1
                    break

        return mask
    
    def getDropRequestMask(self, routeNPArray):
        mask = np.zeros((CAPACITY,), dtype=np.int8)

        for i in range(routeNPArray.shape[0]):
            if routeNPArray[i][0] != PADDINGVAL and routeNPArray[i][1] != PADDINGVAL: #If not padding
                if routeNPArray[i][STARTING_OBSERVATION_INDEX] == STARTINGCORD:
                    mask[i] = 1
                else:
                    mask[i] = 0
            else:
                mask[i] = 0
        return mask
    
    def getCurrentRequest(self):
        return self.currentRequest
    
    def getDropRequestEmpty(self):
        return self.len(self.dropRequests) == 0
    
    def rejectRequest(self):
        self.rejectedRequests.append(self.currentRequest) 
    
    def handleNextDroppedRequests(self):
        req = self.dropRequests.pop(0)
        self.currentRequest = req
    
    def handleRejectedDroppedRequest(self):
        #Find the closest bus to the request
        smallestVal = 10000000
        smallestIndex = 1000000
        for i,bus in enumerate(self.vehiles):
            if bus.getPassengerAmount()+self.currentRequest.getPassengerAmount() <= CAPACITY:
                distance = getDistanceBetweenTwoPoints(bus.currentPosition, self.currentRequest.getOrigin())
                if distance < smallestVal:
                    smallestVal = distance
                    smallestIndex = i
        
        #Add the request to the closest bus
        self.addRequestToRoute(smallestIndex, self.currentRequest)
        self.dropRejectedRequests.append(self.currentRequest)
        
busRouting = BusRoutingHandler()
print(busRouting)

 Vehiles: 
Vehicle: id: 2734551435216, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551438160, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551438864, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551438544, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551438992, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551440400, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551440784, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551442640, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551435088, capacity: 5, currentPosition: lat: 35.8942679,long: 14.5086503, route: 0
Vehicle: id: 2734551443088, capacity: 5, currentPosition: lat: 35.8942679,long: 

In [ ]:
class agentSelector(): #Using a custom Selector because it is more customizable to use a custom selector
    def __init__(self,agents):
        self.agentsList = []
        self.currentAgent = 0
        for agent in agents:
            self.agentsList.append(agent)
        self.possibleAgents = self.agentsList.copy()
    
    def reset(self):
        self.currentAgent = 0
        self.agentsList = self.possibleAgents.copy()
        return self.agentsList[self.currentAgent]
    
    def next(self):
        nextVal = self.currentAgent+1
        if nextVal > len(self.agentsList)-1:
            nextVal = 0
            return True #Means that the agent selector has finished an entire loop
        
    def getCurrentIndex(self): #Returns the index of the current index
        return self.currentAgent

    def getAgentIndex(self,agent): #Returns the index of the agent in the agentsList
        return self.agentsList.index(agent)
    
    def is_last(self):
        return self.currentAgent == len(self.agentsList)-1
    
    def is_first(self):
        return self.currentAgent == 0

In [7]:
def env(render_mode=None): #Petting Zoo Env
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """

    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)

    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)

    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)

    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)

    return env

class raw_env(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "Flexible_Bus"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - render_mode

        Note: as of v1.18.1, the action_spaces and observation_spaces attributes are deprecated.
        Spaces should be defined in the action_space() and observation_space() methods.
        If these methods are not overridden, spaces will be inferred from self.observation_spaces/action_spaces, raising a warning.

        These attributes should not be changed after initialization.
        """

        self.busHandler =  BusRoutingHandler() #Initialize the Bus Routing Handler

        self.agents = ["bus_" + str(r) for r in range(NUMBEROFBUSES)]
        self.possible_agents = self.agents[:]

        #Set up action space and observation space
        self._action_spaces = {
            agent: Dict({
                        "handle_request": Discrete(3), #Requrires Masking to not allow accpet when the request passengers would exceed bus capacity; 0 = Accept, 1 = Reject, 2 = Drop
                        "drop_request": Discrete(CAPACITY), #Require Maskings to allow ensure that a proper request is dropped
                         }) for agent in self.possible_agents
        }
        
        self._observation_spaces = {
            agent: Dict({
                "observation": Dict(
                    {
                    "bus_position": Box(low=0, high=90, shape=(2,), dtype=np.float32), #Simple Defination containing the current bus position cordinates
                    "bus_capacity": Discrete(CAPACITY), #Observation space of the current capacity of the bus
                    "request_position":Box(low=0, high=90, shape=(2,2,2), dtype=np.float32), #The request position cordinates
                    "request_passengers": Discrete(CAPACITY), #The maximum number of passengers allowed in a request
                    "current_route": Box(low=0, high=90, shape=(CAPACITY,4), dtype=np.float32), #The current route of the bus - 4D Array - 0 = Latitude, 1 = Longitude, 2 = Start, 3 = RequestID
                    "routeDistance": Box(low=0, high=np.inf, shape=(1,), dtype=np.float32) #The distance of the current route
                    }),
                "action_mask": Dict(
                    { #Action mask needed to not allow the bus to exceed capacity
                    "handle_request": Box(low=0, high=1, shape=(3,), dtype=np.int8), #0 = Accept, 1 = Reject, 2 = Drop
                    "drop_request": Box(low=0, high=1, shape=(CAPACITY,), dtype=np.int8), #0 = Accept, 1 = Reject
                    })
            }) for agent in self.possible_agents
        } 

        self.agent_selector = agentSelector(self.agents) #Initialize the agent selector
        self.agent_selection = self.agent_selector.reset() #Select the first agent

        self.render_mode = render_mode

    # Observation space defination
    def observation_space(self, agent):
        return self.observation_spaces[agent]

    # Action space defination
    def action_space(self, agent):
        return self.action_spaces[agent]

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        return None

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """

        #Select current agent being observed
        currAgentIndex = self.agent_selector.getAgentIndex(agent) #Get the index of the current agent
        
        #Define the observation of an Agent - Done for more readable code
        observation = {
            "bus_position": np.zeros(2,dtype=np.float32), 
            "bus_capacity": 0,
            "request_position": np.zeros(2,2,2,dtype=np.float32),
            "request_passengers": 0,
            "current_waypoints": np.zeros((CAPACITY,4,),dtype=np.float32),
            "routeDistance": 0
        }

        observation["bus_position"] = BusRoutingHandler.getBusCords(currAgentIndex) #Get the bus position
        observation["bus_capacity"] = self.busHandler.getBusCapacity(currAgentIndex) #Get the bus capacity
        observation["request_position"] = self.busHandler.getCurrentRequestPosition()
        observation["request_passengers"] = self.busHandler.getCurrentRequestPassengerAmount()
        observation["current_waypoints"] = self.busHandler.getCurrentRoute(currAgentIndex)
        observation["routeDistance"] = self.busHandler.getRouteDistance(currAgentIndex)

        #Define the action mask of an Agent - Done for more readable code
        action_mask = {
            "handle_request": np.zeros(2,dtype=np.int8),
            "drop_request": np.zeros(CAPACITY, dtype=np.int8)
        }

        action_mask["handle_request"] = self.busHandler.getAcceptRequestMask(currAgentIndex, self.requestAcceptedFlag, observation["request_passengers"], observation["current_waypoints"])
        action_mask["drop_request"] = self.busHandler.getDropRequestMask(observation["current_waypoints"])
        return {"observation": observation, "action_mask": action_mask}

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.dones = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        #Selects the first bus
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

        #Not sure about this state
        self.state = {agent: 0 for agent in self.agents}

        #Custom Variables
        self.busHandler =  BusRoutingHandler() #Initialize the Bus Routing Handler
        self.requestAcceptedFlag = False
        self.handleDroppedRequest = False

        """
        Need to add a bit where the first request is handled
        """

    #Requries Definations
    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            self._was_dead_step(action)
            return

        agent = self.agent_selection

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        self._cumulative_rewards[agent] = 0

        # stores action of current agent
        self.state[self.agent_selection] = action            

        if action["handle_request"] == REQUEST_ACCEPTED:
            #Handle the accepted request
            self.requestAcceptedFlag = True
            #Add the request to the bus route
            self.busHandler.addRequestToRoute(self.agent_selector.getCurrentIndex(), self.busHandler.getCurrentRequest())

        elif action["handle_request"] == REQUEST_DROPPED:
            self.busHandler.dropRequest(self.agent_selector.getCurrentIndex(), action["drop_request"])
        
        elif action["handle_request"] == REQUEST_REJECTED:
            pass #Do Nothing
            
        
        #Go through a set of steps: 1. Check the dropped list, update the route, distribute the rewards. 
        if self._agent_selector.is_last():
            if self.requestAcceptedFlag == False:

                if self.handleDroppedRequest == True:
                    self.busHandler.handleRejectedDroppedRequest()
                    self.handleDroppedRequest = False

                self.busHandler.rejectRequest() #Add the request to the rejected list

            if self.busHandler.getDropRequestEmpty() == True:
                self.handleDroppedRequest = False
                #Distribute the rewards

                #Update the route
                self.busHandler.updateRequest()

                #Get Cumulitive Rewards
            else:
                #Switch the current request with Dropped Requests
                self.busHandler.handleNextDroppedRequests()
                self.handleDroppedRequest = True
                self._clear_rewards()
        else:
            #Adjsut the observation to be a reasonable one 
            #self.state[self.agents[1 - self.agent_name_mapping[agent]]] = NONE
            #No rewards are allocated until all buses take an action
            self._clear_rewards()
            pass

        #Selects the next agent.
        self.agent_selection = self._agent_selector.next()

        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        #Render Step if human
        if self.render_mode == "human":
            self.render()